In [1]:
from datetime import datetime
import spotipy
import billboard
#from sklearn.ensemble import RandomForestClassifier
#from sklearn.model_selection import train_test_split
import time
import pandas as pd
#import numpy as np
from spotipy.oauth2 import SpotifyClientCredentials
cid = "b3c5b350c6cb43c2bdb403eb3b896eba"
secret = "fa1aa37ba9c943d3b4e2cadc8fd4818f"
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)


In [2]:
def getTrackIDs(user, playlist_id):
    ids = []
    playlist = sp.user_playlist(user, playlist_id)
    for item in playlist['tracks']['items']:
        track = item['track']
        ids.append(track['id'])
    return ids

In [3]:
def getTrackIDsFromAlbum(album_id):
    ids2 = []
    #playlist = sp.user_playlist(user, playlist_id)
    album = sp.album(album_id)
    for item in album['tracks']['items']:
        ids2.append(item['id'])
    return ids2

In [4]:
def getTrackFeatures(id):
      meta = sp.track(id)
      features = sp.audio_features(id)

      # meta
      song_id = id
      name = meta['name']
      artist = meta['album']['artists'][0]['name']
      length = meta['duration_ms']
      popularity = meta['popularity']
      release_date = meta['album']['release_date']

      # features
      acousticness = features[0]['acousticness']
      danceability = features[0]['danceability']
      energy = features[0]['energy']
      instrumentalness = features[0]['instrumentalness']
      liveness = features[0]['liveness']
      loudness = features[0]['loudness']
      speechiness = features[0]['speechiness']
      tempo = features[0]['tempo']
      time_signature = features[0]['time_signature']
      valence = features[0]['valence']
      mode = features[0]['mode']
      weeks = 1
      chart_date = 1
      target = 1

      track = {"name" : name, "artist" : artist, "song_id" : song_id, "length" : length, "popularity" : popularity, 
               "danceability" : danceability, "acousticness" : acousticness, "energy" : energy, "instrumentalness" : instrumentalness, 
               "liveness" : liveness, "loudness" : loudness, "speechiness" : speechiness, "tempo" : tempo, "time_signature" : time_signature, 
               "valence" : valence, "mode" : mode, "weeks" : weeks, "chart_date" : chart_date, "release_date" : release_date, "target" : target}
      return track

In [5]:
def getTrackIdFromBillboard(name, artist):
    results = sp.search(q="track:" + name + " artist:" + artist)
    items = results["tracks"]["items"]
    id = "NONE"
    if (len(items) > 0):
        id = items[0]["id"]
    return id

In [6]:
def getChartDataFrame(chart, fullDataFrame):
    tracks = []
    for entry in chart.entries:
        id = getTrackIdFromBillboard(entry.title, entry.artist)

        if (id != "NONE"):
            if (entry.artist in fullDataFrame.values) and (not any(dic['artist'] == entry.artist for dic in tracks)):
                fullDataFrame.loc[fullDataFrame['artist'] == entry.artist, 'weeks'] += 1
            if (id not in fullDataFrame.values) and (entry.isNew):
                print(entry)
                track = getTrackFeatures(id)
                track["chart_date"] = datetime.strptime(chart.date, '%Y-%m-%d').date()
                tracks.append(track)
    df = pd.DataFrame(tracks)
    fullDataFrame = fullDataFrame.append(df)
    return fullDataFrame

In [7]:
def gatherBillboardData():
    chart = billboard.ChartData('hot-100')
    i = 0
    fullDataFrame = pd.DataFrame(columns = ['name', 'artist', 'song_id', 'length', 'popularity', 'danceability', 'acousticness', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo', 'time_signature', 'valence', 'mode', 'weeks', 'release_date', 'chart_date'])
    start = time.time()
    while (chart.previousDate):
        #start = time.clock()
        fullDataFrame = getChartDataFrame(chart, fullDataFrame)
        chart = billboard.ChartData('hot-100', chart.previousDate)
        #end = time.clock()
        #print("total time: " + str(end-start))
        i += 1
        if (i == 1): #was 960
            break
        if (i == 480):
            print("halveis")
    end = time.time()
    print("total time: " + str(end-start))
    return fullDataFrame
from pathlib import Path
data_folder = Path("CSV-files")
writeFile = "testbillboard.csv"
writePath = data_folder / writeFile

In [8]:
df = gatherBillboardData()
#print(df)
#df.to_csv(writeFile, index=False, header=True)
#dataFrame.head(40)
#print(len(dataFrame))
#date1 = datetime.strptime('2020-10-17', '%Y-%m-%d').date()
#dataFrame.loc[(dataFrame['artist'] == '')]


hot-100 chart from 2020-11-14
-----------------------------
1. 'Mood' by 24kGoldn Featuring iann dior
2. 'Positions' by Ariana Grande
3. 'Laugh Now Cry Later' by Drake Featuring Lil Durk
4. 'Blinding Lights' by The Weeknd
5. 'I Hope' by Gabby Barrett Featuring Charlie Puth
6. 'Savage Love (Laxed - Siren Beat)' by Jawsh 685 x Jason Derulo
7. 'Lemonade' by Internet Money & Gunna Featuring Don Toliver & NAV
8. '34+35' by Ariana Grande
9. 'Dakiti' by Bad Bunny & Jhay Cortez
10. 'WAP' by Cardi B Featuring Megan Thee Stallion
11. 'Holy' by Justin Bieber Featuring Chance The Rapper
12. 'For The Night' by Pop Smoke Featuring Lil Baby & DaBaby
13. 'Rockstar' by DaBaby Featuring Roddy Ricch
14. 'Before You Go' by Lewis Capaldi
15. 'Go Crazy' by Chris Brown & Young Thug
16. 'Dynamite' by BTS
17. 'Watermelon Sugar' by Harry Styles
18. 'Forever After All' by Luke Combs
19. 'Kings & Queens' by Ava Max
20. 'Said Sum' by Moneybagg Yo
21. 'Lonely' by Justin Bieber & benny blanco
22. 'Be Like That' by K